# Let's set light to the place!

- https://adventofcode.com/2023/day/16

This is, at a first glance, a typical "follow the maze" problem. Some notes on the specifics:

- Because beams can be split into two, we'll need a queue to which we can add additional beams to process.
- It's likely beams go round in circles, so we need to stop processing beams that end up at a position that they already visited.
- We need to make sure to not only mark what locations we have already lit up, but also what direction of beam was flowing when we got there.

I'm using complex numbers again to make position tracking easy. I've implemented the contraption tile placement as a dictionary of (complex) positions to tily types, so we can simply test for dictionary containment to see if a position is legal, this saves on boundary checks. To track what positions have been lit up, I'm keeping a set of position and beam pairs; the final answer is the unique positions from this set.


In [1]:
from __future__ import annotations

import typing as t
from collections import deque
from dataclasses import dataclass, field
from enum import IntEnum, StrEnum

type Pos = complex


class Beam(IntEnum):
    # value, delta
    up = 0, -1j
    right = 1, 1
    down = 2, 1j
    left = 3, -1

    if t.TYPE_CHECKING:
        delta: Pos
    else:
        # keep the new method hidden from the type checker so they only
        # retain the standard Enum() call interface.
        def __new__(cls, value: int, delta: Pos) -> "Beam":
            inst = int.__new__(cls, value)
            inst._value_ = value
            inst.delta = delta
            return inst


type Effect = tuple[()] | tuple[Beam] | tuple[Beam, Beam]
_hor: Effect = (Beam.left, Beam.right)
_ver: Effect = (Beam.up, Beam.down)


class LightTile(StrEnum):
    # value, up, right, down, left
    empty = ".", (Beam.up, Beam.right, Beam.down, Beam.left)
    mirror_bltr = "/", (Beam.right, Beam.up, Beam.left, Beam.down)
    mirror_brtl = "\\", (Beam.left, Beam.down, Beam.right, Beam.up)
    splitter_hor = "-", (_hor, Beam.right, _hor, Beam.left)
    splitter_ver = "|", (Beam.up, _ver, Beam.down, _ver)

    if t.TYPE_CHECKING:
        # indices correspond with the beam int values 0 - 3
        effects: tuple[Effect, Effect, Effect, Effect]
    else:
        # keep the new method hidden from the type checker so they only
        # retain the standard Enum() call interface.
        def __new__(
            cls,
            value: str,
            effects: tuple[Beam | Effect, Beam | Effect, Beam | Effect, Beam | Effect],
        ) -> "LightTile":
            inst = str.__new__(cls, value)
            inst._value_ = value
            inst.effects = tuple(e if isinstance(e, tuple) else (e,) for e in effects)
            return inst


@dataclass
class Contraption:
    layout: dict[Pos, LightTile]
    width: int
    height: int

    @classmethod
    def from_text(cls, text: str) -> t.Self:
        lines = text.splitlines()
        layout = {
            complex(x, y): LightTile(c)
            for y, line in enumerate(lines)
            for x, c in enumerate(line)
        }
        return cls(layout, len(lines[0]), len(lines))

    # all lit positions from a given splitter
    _cache: dict[Pos, set[tuple[Pos, Beam]]] = field(default_factory=dict, repr=False)

    def energized_tiles(self, start: Pos = -1, beam: Beam = Beam.right) -> int:
        layout, cache = self.layout, self._cache
        lit: set[tuple[Pos, Beam]] = set()
        todo: deque[tuple[Pos, Beam]] = deque([(start, beam)])
        # uncached split we have passed so far, to update cache for
        splits: list[Pos] = []
        while todo:
            pos, beam = todo.popleft()
            pos += beam.delta
            if (pos, beam) in lit or (tile := layout.get(pos)) is None:
                continue
            lit.add((pos, beam))
            for spos in splits:
                cache.setdefault(spos, set()).add((pos, beam))
            if len(tile.effects[beam]) == 2:
                # beam is being split; see if there is a cached result from this
                # point forward. If so, we are done with this beam, otherwise
                # add this split to the list of splits to maintain the cache
                # for.
                if (cached := cache.get(pos)) is not None:
                    lit.update(cached)
                    for spos in splits:
                        cache[spos].update(cached)
                    continue
                splits.append(pos)
            for direction in tile.effects[beam]:
                todo.append((pos, direction))
        return len({pos for pos, _ in lit})


test_layout = r"""
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
""".lstrip()
test_conraption = Contraption.from_text(test_layout)

assert test_conraption.energized_tiles() == 46

In [2]:
import aocd

layout = aocd.get_data(day=16, year=2023)
contraption = Contraption.from_text(layout)
print("Part 1:", contraption.energized_tiles())

Part 1: 7392


## Part 2, it's getting awfully bright in here!

For part 2, I refactored part 1 to cache partial beam paths; otherwise visiting every possible starting point would do a lot of redudant work and solving this part would take too long.

The cache stores the full set of visited positions _per splitter_; when a beam reaches a splitter that splits that beam, we can immediately update the `lit` set with the additional positions resulting from that split. Thanks to this cache, part 2 completes in less than half a second.


In [3]:
def maximize_configuration(contraption: Contraption) -> int:
    width, height = contraption.width, contraption.height
    best = 0
    # from the left and the right edges
    for y in range(height):
        for x, beam in ((-1, Beam.right), (width, Beam.left)):
            best = max(best, contraption.energized_tiles(complex(x, y), beam))
    # from the top and bottom edges
    for x in range(width):
        for y, beam in ((-1, Beam.down), (height, Beam.up)):
            best = max(best, contraption.energized_tiles(complex(x, y), beam))
    return best


assert maximize_configuration(test_conraption) == 51

In [4]:
print("Part 2:", maximize_configuration(contraption))

Part 2: 7665
